In [147]:
import os
import matplotlib.pyplot as plt
from astropy.io import fits
from matplotlib.backends.backend_pdf import PdfPages
import numpy as np
from astropy.coordinates import SkyCoord
from astropy import units as u
from astropy.cosmology import FlatLambdaCDM

In [148]:
mpl8_dir = 'local/'
drp = fits.open(mpl8_dir + 'drpall-v2_5_3.fits')
drpdata = drp[1].data 
stellar_mass = drpdata.field('nsa_sersic_mass')

In [149]:
# getting MPA-JHU catalog information
gal = fits.open(mpl8_dir + 'gal_info_dr7_v5_2.fit')
galdata = gal[1].data
sfr = fits.open(mpl8_dir + 'gal_totsfr_dr7_v5_2.fits')
sfrdata = sfr[1].data
mass = fits.open(mpl8_dir + 'gal_totsfr_dr7_v5_2.fits')
massdata = mass[1].data

In [150]:
# getting information from NSA catalog
nsa = fits.open(mpl8_dir + '1-nsa_v1_0_1.fits')
nsa_data = nsa[1].data

In [151]:
# matching the MPA-JHU and MaNGA catalogs on RA and DEC
manga_ra = drpdata.field('objra')
manga_dec = drpdata.field('objdec')
mpa_ra = galdata.field('RA')
mpa_dec = galdata.field('DEC')
bad = mpa_dec < -90
mpa_dec[bad] = -90
mpa_cat = SkyCoord(ra=mpa_ra*u.degree, dec=mpa_dec*u.degree)
c = SkyCoord(ra=manga_ra*u.degree, dec=manga_dec*u.degree)
idx_mpa, d2d_mpa, d3d_mpa = c.match_to_catalog_sky(mpa_cat) 

In [152]:
sfr_manga = sfrdata.field('MEDIAN')[idx_mpa]
re_arcsec = drpdata.field('nsa_elpetro_th50_r')#nsa_data.field('ELPETRO_TH50_R')
cosmo = FlatLambdaCDM(H0=70 * u.km / u.s / u.Mpc, Om0=0.3)
z = drpdata.field('nsa_z')
print(len(re_arcsec))
len(z)
#drpdata

6779


6779

In [153]:
re_kpc = re_arcsec * u.arcsec / cosmo.arcsec_per_kpc_proper(z)
sigma_sfr = 10**sfr_manga / (2 * np.pi * re_kpc**2)

/Users/rileytheriault/anaconda3/lib/python3.7/site-packages/astropy/cosmology/core.py:1922: RuntimeWarning: invalid value encountered in sqrt
  return 2 * np.sqrt(x) * hyp2f1(1./6, 1./2, 7./6, -x**3)


In [154]:
print(re_kpc)

[ 4.01112287  5.65680643  5.91935481 ...  4.21399515 10.6887845
  3.49211062] kpc


In [155]:
print(sigma_sfr)
sort = np.argsort(sigma_sfr)
print(sort[6300:6400])
print(sigma_sfr[sort][6300:6400])

[0.00936724 0.00145146 0.00743165 ... 0.02340948 0.00030319 0.01142684] 1 / kpc2
[6450 3402 5837 3403 6001 5242 4891 4902 3365 6106 3033  176 5058  934
  976 3118 1447 3742  910 6650 4831 3851 2284 6395 4939 4085 4173 4553
  967 5937 6484 5170 4899 5123 2969 4886 5737 4804 5680 3170 5363 1774
  302 5938 5868 2616 6734 2639 6764 6771 6311 2278 1524 3819 1957  689
  213 1838 1855  810 1656 5431 2745 6029 4915 5631 4141  211 3732 4746
 6690 4642  348 4798 4850 6683 1261 4184 2928 5341 1674 3337 1809 1231
 2671 4392 6020 3202 1602 3578 5308 4475 1176 1009 3557 3058  231 5179
 5192 1193]
[0.06605533 0.06735494 0.06750741 0.06773563 0.06774775 0.06784296
 0.06871549 0.06872746 0.06887275 0.06897282 0.06967202 0.07001213
 0.0701695  0.070444   0.07084638 0.0711444  0.07121488 0.07146561
 0.07174506 0.07236975 0.0726841  0.07310981 0.07361492 0.07378026
 0.07390901 0.07403148 0.07543778 0.07602351 0.07646497 0.07664408
 0.07754349 0.07782598 0.07797624 0.0781509  0.07967443 0.0797261
 0.080186

In [156]:
drpdata.field('plateifu')[sort][6300:6400]

chararray(['9875-6101', '8615-1901', '9497-3703', '8615-1902',
           '9507-12704', '9086-3703', '9032-9101', '9033-3703',
           '8613-12703', '9513-1902', '8553-3704', '10215-1902',
           '9042-6102', '7993-9101', '8079-1902', '8568-3704',
           '8153-1901', '8719-1901', '7992-3702', '9887-3701',
           '9029-1902', '8725-6102', '8335-3703', '9872-3701',
           '9035-6102', '8941-3702', '8946-6101', '8992-12701',
           '8078-6104', '9503-3701', '9877-6101', '9049-1901',
           '9033-1902', '9046-3703', '8549-6104', '9032-3704',
           '9491-6101', '9027-6103', '9488-1901', '8592-6101',
           '9094-6101', '8257-3703', '10492-6103', '9503-3702',
           '9499-1902', '8454-12702', '9892-1902', '8455-3701',
           '9894-12703', '9894-3703', '9867-3702', '8335-12704',
           '8158-6104', '8723-6104', '8313-1901', '7815-3702',
           '10217-3703', '8261-1901', '8262-1901', '7972-3704',
           '8249-3704', '9183-6101', '8464-610

In [157]:
sigma_sfr_nonan = sigma_sfr[np.logical_not(np.isnan(sigma_sfr))]
ten = np.percentile(sigma_sfr_nonan,10)
print(ten)
ninety = np.percentile(sigma_sfr_nonan,90)
print(ninety)
gal_high_sigma_sfr = (sigma_sfr*u.kpc*u.kpc > ninety)
print(sigma_sfr[gal_high_sigma_sfr])
drpdata.field('plateifu')[gal_high_sigma_sfr]

0.0002770038881975685
0.022082837804183623
[0.03336283 0.0380081  0.04966357 0.07001213 0.03516188 0.02665664
 0.02897323 0.02336711 0.04082435 0.09241352 0.08693034 0.03768313
 0.13200421 0.03553528 0.12172886 0.02793615 0.02484628 0.04362451
 0.0258316  0.027155   0.03940934 0.08166    1.37274915 0.04444228
 0.03188586 0.14615598 0.09587135 0.04510229 0.02644545 0.02490496
 1.87719804 0.02824456 0.04985047 0.04903588 0.03159002 0.05114918
 0.05272612 0.06037592 0.02236062 0.04003528 0.02900742 0.02619259
 0.02446829 0.02860682 0.02251016 0.05006546 0.03251568 0.13421846
 0.20520328 0.02247933 0.02218351 0.08602274 0.02775117 0.0428255
 0.02917417 0.02320916 0.03964969 0.08747807 0.02250507 0.03617977
 0.13849083 0.20555967 0.13777384 0.03465038 0.07174506 0.03127103
 0.04803794 0.25303597 0.05225846 0.070444   0.03738403 0.02514844
 0.0249714  0.07646497 0.04250359 0.07084638 0.06090177 0.02608024
 0.03018426 0.11401257 0.02247129 0.05851859 0.0240117  0.03020136
 0.02854102 0.054617

/Users/rileytheriault/anaconda3/lib/python3.7/site-packages/astropy/units/quantity.py:463: RuntimeWarning: invalid value encountered in greater
  result = super().__array_ufunc__(function, method, *arrays, **kwargs)


chararray(['10001-3702', '10001-3703', '10214-1902', '10215-1902',
           '10215-3704', '10215-6103', '10215-6104', '10216-6101',
           '10216-6103', '10217-3701', '10217-3703', '10217-6103',
           '10218-1901', '10218-1902', '10218-3704', '10218-6102',
           '10218-6103', '10219-12702', '10220-3703', '10221-3704',
           '10221-6102', '10492-6103', '10492-9101', '10493-3704',
           '10494-3701', '10496-12704', '10496-3702', '10497-6101',
           '10498-3704', '10499-3702', '10503-3702', '10504-12701',
           '10504-9101', '10506-1901', '10507-1901', '10508-1901',
           '10509-12703', '10509-1901', '10509-3703', '10509-3704',
           '10509-6101', '10512-1902', '10516-12701', '10516-1902',
           '10516-3702', '10517-3701', '10517-3704', '10517-6102',
           '7443-12703', '7443-3703', '7495-3701', '7815-3702',
           '7815-6103', '7958-6101', '7962-6104', '7962-9101',
           '7968-3704', '7972-3704', '7977-12701', '7977-3703',


In [158]:
ba = drpdata.field('nsa_sersic_ba')

# get things to make that c_manga thing
nsa_ra = nsa_data.field('RA')
nsa_dec = nsa_data.field('DEC')
catalog = SkyCoord(ra=nsa_ra*u.degree, dec=nsa_dec*u.degree)
idx, d2d, d3d = c.match_to_catalog_sky(catalog) 

th50 = nsa_data.field('ELPETRO_TH50_R')
th90 = nsa_data.field('ELPETRO_TH90_R')
th50_manga = th50[idx]
th90_manga = th90[idx]
c_manga = th90_manga / th50_manga

# I think this one is good?

In [159]:
# then define things that are late type (concentration) then edge-on (axis ratio)
blah = drpdata.field('srvymode') == 'MaNGA dither'
print(len(c_manga[blah]))

# getting the gals
c_manga_gal = c_manga[blah]
ba_gal = ba[blah]

#getting the late-type ones
late = c_manga_gal < 2.6

ba_gal_late = ba_gal[late]
c_manga_gal_late = c_manga_gal[late]

print(len(c_manga_gal_late))
# ok so after that business then do what we did about to get the ten percent of the late type, edge-on galaxies and I 
# should ghet about sixty of those. After that, make a slideshow like the one I did earlier with my special galaxies 
# to compare their maps. neato.

6661
3033


In [160]:
edge = ba_gal_late < 0.3

# update concentration and ba arrays
c_manga_gal_late_edge = c_manga_gal_late[edge]
ba_gal_late_edge = ba_gal_late[edge]

print(len(c_manga_gal_late_edge))

647


In [161]:
sigma_sfr_us = sigma_sfr[blah][late][edge]
good = np.logical_not(np.isnan(sigma_sfr_us))
#print(sigma_sfr_us)
#sigma_sfr_awesome = sigma_sfr_us[np.logical_not(np.isnan(sigma_sfr_us))]
sigma_sfr_awesome = sigma_sfr_us[good]
print(len(sigma_sfr_us), len(sigma_sfr_awesome))

647 596


In [162]:
ten = np.percentile(sigma_sfr_awesome,10)#c_manga_gal_late_edge,10)
print(ten)
ninety = np.percentile(sigma_sfr_awesome,90)#c_manga_gal_late_edge,90)
print(ninety)
gal_high_c_manga_gal_late_edge = (sigma_sfr_awesome*u.kpc*u.kpc > ninety)
#print(c_manga_gal_late_edge[gal_high_c_manga_gal_late_edge])
#print(gal_high_c_manga_gal_late_edge)
plateifu_high = drpdata.field('plateifu')[blah][late][edge][good][gal_high_c_manga_gal_late_edge]
print(plateifu_high)
print(len(plateifu_high))
sigma_sfr_high = sigma_sfr_awesome[gal_high_c_manga_gal_late_edge]
print(sigma_sfr_high)

0.0005706972328089633
0.011793797735198196
['10220-6104' '7960-12702' '7960-6103' '7979-12701' '8077-9101'
 '8081-1901' '8083-3703' '8085-3701' '8086-3704' '8137-3704' '8138-6101'
 '8138-6103' '8141-3701' '8144-12705' '8249-1901' '8252-9101' '8260-6101'
 '8323-6103' '8324-12701' '8324-6103' '8325-3701' '8331-3703' '8332-3704'
 '8332-9102' '8439-6103' '8442-12701' '8453-1902' '8458-3701' '8465-3703'
 '8465-9101' '8547-9102' '8551-6104' '8600-6104' '8601-6103' '8603-6103'
 '8624-12702' '8711-1902' '8716-6102' '8939-6101' '8943-1901' '8950-1901'
 '8981-3703' '8992-12701' '8993-6104' '8999-3704' '9031-12704' '9035-3702'
 '9047-12705' '9086-6102' '9089-3704' '9090-1902' '9091-12704' '9184-3704'
 '9185-3704' '9190-3703' '9192-1901' '9193-9102' '9485-3701' '9891-6102'
 '9894-12703']
60
[0.02087265 0.02184589 0.01670782 0.01322442 0.01228151 0.11401257
 0.01677868 0.01476489 0.02052567 0.01302549 0.01659246 0.0153078
 0.01401969 0.01199777 0.04961361 0.01602484 0.02819373 0.03314391
 0.0337286

In [163]:
ten = np.percentile(sigma_sfr_awesome,10)
print(ten)
gal_high_c_manga_gal_late_edge_10 = (sigma_sfr_awesome*u.kpc*u.kpc < ten)
plateifu_10 = drpdata.field('plateifu')[blah][late][edge][good][gal_high_c_manga_gal_late_edge_10]
print(plateifu_10)
print(len(plateifu_10))
sigma_sfr_high_10 = sigma_sfr_awesome[gal_high_c_manga_gal_late_edge_10]
print(sigma_sfr_high_10)

0.0005706972328089633
['10213-12703' '10503-12703' '10510-9101' '10512-9102' '10517-12701'
 '7958-12703' '7958-12704' '7977-12704' '7979-6102' '7990-12702'
 '7991-6102' '8078-12702' '8086-12702' '8097-12701' '8145-12705'
 '8149-12701' '8158-12701' '8244-12705' '8249-12702' '8249-9102'
 '8259-12702' '8260-12705' '8261-12704' '8312-6101' '8446-12703'
 '8450-12703' '8452-12702' '8464-12703' '8549-12704' '8597-6101'
 '8615-12702' '8618-12701' '8712-9102' '8713-3701' '8714-1901' '8931-6101'
 '8932-12702' '8935-12701' '8938-12705' '8945-12704' '8977-12704'
 '8988-12702' '8988-1901' '8994-12704' '9050-12703' '9086-12701'
 '9090-12703' '9181-12705' '9186-12703' '9491-12702' '9495-12705'
 '9496-12703' '9497-12702' '9499-12704' '9502-12704' '9502-12705'
 '9505-3703' '9870-12704' '9871-1902' '9885-12701']
60
[3.24287109e-04 8.07158552e-05 5.02229364e-04 4.59413370e-04
 3.21317444e-04 5.54518825e-04 2.10196939e-04 5.85942819e-05
 5.05359060e-04 3.64957612e-04 3.94645270e-04 1.47712114e-04
 2.17495